In [1]:
#r "nuget: Jinaga"
#r "nuget: Jinaga.Store.SQLite"
#r "nuget: Jinaga.Notebooks"
#r "nuget: Jinaga.UnitTest"

using Jinaga;
using Jinaga.Notebooks;
using Jinaga.Store.SQLite;
using Jinaga.UnitTest;
using System.Collections.Immutable;

Installed Packages Jinaga, 1.1.39 Jinaga.Notebooks, 1.1.8 Jinaga.Store.SQLite, 1.1.10 Jinaga.UnitTest, 1.1.4

Loading extensions from `/Users/michaelperry/.nuget/packages/microsoft.data.analysis/0.22.0/interactive-extensions/dotnet/Microsoft.Data.Analysis.Interactive.dll`

# Jinaga Quick Start

Jinaga is a data management framework for mobile, web, and distributed applications.
It is particularly good for offline-first applications and progressive web apps, where data is stored locally and synchronized with a server.
We call that server a *replicator*, but we'll get to that.

## Jinaga Client

A Jinaga client manages data in your application and connects to a replicator.
Create a client using a factory method.
For example, to store local state in SQLite, use this method:

In [2]:
JinagaClient j = JinagaSQLiteClient.Create(options =>
{
    // Connect to the Jinaga replicator that you host yourself, or that we host for you.
    options.HttpEndpoint = new Uri("https://rep.jinaga.com/myreplicator");

    // Use the SQLite store to persist data locally.
    options.SQLitePath = System.IO.Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.LocalApplicationData),
        "jinaga", "myapplication.sqlite"
    );
});

Or to use Jinaga in a unit test, use an in-memory store:

In [3]:
JinagaClient j = JinagaTest.Create(options =>
{
    options.User = new User("--- TEST USER ---");
});

## Facts

The core unit of data in Jinaga is a *fact*.
A fact is an immutable record of an event, decision, or state change.
Let's start simple.
When the user logs into your application, that's a fact.

In [4]:
(User user, UserProfile profile) = await j.Login();

j.RenderFacts(user)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER ---

Each fact has a type (`Jinaga.User` in this case) and a set of properties (`publicKey` for this example).

The `Jinaga.User` type is defined in the Jinaga library.
But, of course, you can define your own types.
Write a record and use the `FactType` attribute to define a type.

In [5]:
[FactType("Construction.Project")]
public record Project(User creator, Guid id);

To create an instance of this fact, call `j.Fact`.
This method is asynchronous because it will save the fact to the local store and notify the replicator to synchronize it with other clients.

In [6]:
Project projectA = await j.Fact(new Project(user, Guid.NewGuid()));

j.RenderFacts(projectA)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

## Predecessors

When we defined the `Construction.Project` type, we used `Jinaga.User` as one of its properties.
That made it a *predecessor*.
In this case, the meaning of that predecessor relationship is that the user created the project.

Once you have predecessors, you can query for facts that are related to them.
These, as you might have guessed, are called *successors*.
You'll find the `Successors` extension function in the `Jinaga.Extensions` namespace.

Let's query for all projects that a user has created.

In [7]:
using Jinaga.Extensions;

// Create a couple more projects.
Project projectB = await j.Fact(new Project(user, Guid.NewGuid()));
Project projectC = await j.Fact(new Project(user, Guid.NewGuid()));

var projectsCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
);

ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== 
 
 Construction.Project 
 
 id 
 
 84d2f319-e553-489f-b... 
 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== 
 
 Construction.Project 
 
 id 
 
 9b006289-af52-41eb-b... 
 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

Let's break down that specification.
Start with `Given` and the parameter type.
Then use `Match` to write an expression that matches the facts you want.

The `Successors` extension method finds all successors `OfType<T>` related to the given predecessor.
Provide a lambda that shows how the successors relate to the predecessor.

## Deletion

Facts are immutable, so you cannot really delete them.
Instead, you can create a new fact that marks its predecessor as deleted.

In [8]:
[FactType("Construction.Project.Deleted")]
public record ProjectDeleted(Project project, DateTime deletedAt);

var projectADeleted = await j.Fact(new ProjectDeleted(projectA, DateTime.UtcNow));

On its own, this fact does not do anything.

In [9]:
ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== 
 
 Construction.Project 
 
 id 
 
 84d2f319-e553-489f-b... 
 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== 
 
 Construction.Project 
 
 id 
 
 9b006289-af52-41eb-b... 
 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

But if you include it in the specification, it will filter out the predecessor from the results.

In [10]:
var projectsCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .Where(p => !p.Successors().OfType<ProjectDeleted>(d => d.project).Any())
);

ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== 
 
 Construction.Project 
 
 id 
 
 84d2f319-e553-489f-b... 
 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== 
 
 Construction.Project 
 
 id 
 
 9b006289-af52-41eb-b... 
 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

Now if you are concerned about facts never being truly deleted, there is a way to *purge* unreachable facts.
We'll save that for later.

## Restore

For completion, let's talk about restoring a deleted fact.
You do this by defining yet another successor.

In [11]:
[FactType("Construction.Project.Restored")]
public record ProjectRestored(ProjectDeleted deleted);

await j.Fact(new ProjectRestored(projectADeleted));

Again, you need to add this fact to the specification for it to have an effect.

In [12]:
var projectsCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .Where(p => !p.Successors().OfType<ProjectDeleted>(d => d.project).Any(
            d => !d.Successors().OfType<ProjectRestored>(r => r.deleted).Any()))
);

ImmutableList<Project> projects = await j.Query(projectsCreatedByUser, user);

j.RenderFacts(projects)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA== 
 
 Construction.Project 
 
 id 
 
 84d2f319-e553-489f-b... 
 
 
<!-- KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 KYqFiiUVBpE/Sh46+rqc2kZw5uQ/ya1fjuL9+AS/V/Y6WltTEaoDqb5DqGB2b7BjO8K5n5b/CKn0oMByezWKMA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA== 
 
 Construction.Project 
 
 id 
 
 9b006289-af52-41eb-b... 
 
 
<!-- odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 odlVShw7StqieaN5xmS0hMZnulXscQ1yOLzq+f2VCbfMUtF6VW4ZNVyUL/bzXSdDK7wWfU8tf5ht7LKbtCHCKA==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator

And it's back!

### Pattern Extensions

Because these are such common patterns, Jinaga provides some shorthand extensions to make them easier to write.
You can find these in the `Jinaga.Patterns` namespace.

In [13]:
using Jinaga.Patterns;

var projectsCreatedByUserWithoutRestore = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .WhereNotDeleted((ProjectDeleted d) => d.project)
);

var projectsCreatedByUserWithRestore = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .WhereNotDeletedOrRestored((ProjectDeleted d) => d.project,
            (ProjectRestored r) => r.deleted)
);


## Mutable Properties

Facts are immutable.
So you don't want to put a property like `name` in a project.
You would never be able to change it!

Instead, you can define a successor that represents changing the project name.
This is the pattern we use.

In [14]:
[FactType("Construction.Project.Name")]
public record ProjectName(Project project, string value, ProjectName[] prior);

That `prior` array let's you overwrite a prior value.
The first value will have an empty array, which means there is no prior value.

In [15]:
ProjectName projectAName1 = await j.Fact(new ProjectName(projectA, "House on Cheyenne", []));

j.RenderFacts(projectAName1)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project

To change the name, you create a new fact that has the old name as its prior value.

In [16]:
ProjectName projectAName2 = await j.Fact(new ProjectName(projectA, "House on Rivercrest", [projectAName1]));

j.RenderFacts(projectAName2)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Rivercrest 
 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 
 prior

To query for the current name, you look for the successor that has no prior value.

In [17]:
var namesOfProject = Given<Project>.Match(p =>
    p.Successors().OfType<ProjectName>(n => n.project)
        .Where(p => !p.Successors().OfType<ProjectName>(next => next.prior).Any())
);

ImmutableList<ProjectName> names = await j.Query(namesOfProject, projectA);

names.Select(n => n.value)

[ House on Rivercrest ]

Jinaga provides a shorthand for this pattern as well.

In [18]:
var namesOfProject = Given<Project>.Match(p =>
    p.Successors().OfType<ProjectName>(n => n.project)
        .WhereCurrent((ProjectName next) => next.prior)
);

### Encapsulating Specifications

You can give a specification a name and attach it to a fact type.
Use the `Relation` type to define a property.

```csharp
[FactType("Construction.Project")]
public record Project(User creator, Guid id)
{
    public Relation<ProjectName> Names => Relation.Define(_ =>
        this.Successors().OfType<ProjectName>(n => n.project)
            .WhereCurrent((ProjectName next) => next.prior)
    );
}

var namesOfProject = Given<Project>.Match(p =>
    p.Names
);
```

Since the `Jinaga.User` type is defined in the Jinaga library, you need to use an extension method to add a relation to it.

```csharp
public static class UserExtensions
{
    public static Relation<Project> Projects(this User user) => Relation.Define(_ =>
        user.Successors().OfType<Project>(p => p.creator)
            .WhereNotDeletedOrRestored((ProjectDeleted d) => d.project,
                (ProjectRestored r) => r.deleted));
}

var projectsCreatedByUser = Given<User>.Match(u =>
    u.Projects()
);
```

### Concurrent Edits

The reason that we use the `prior` array is to handle concurrent edits.
If a user on a different device changes the name of the project, you will get a fork in the graph.

In [19]:
ProjectName projectAName3 = await j.Fact(new ProjectName(projectA, "Home on Cheyenne", [projectAName1]));

j.RenderFacts(projectAName1, projectAName2, projectAName3)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Rivercrest 
 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 
 prior 
 
<!-- 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA== -->
 
 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA== 
 
 Construction.Project.Name 
 
 value 
 
 Home on Cheyenne 
 
 
<!-- 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 
 prior

That will result in more than one successor matching the specification.

In [20]:
ImmutableList<ProjectName> names = await j.Query(namesOfProject, projectA);

names.Select(n => n.value)

[ House on Rivercrest, Home on Cheyenne ]

This is how you can recognize that there have been concurrent edits.
It also shows you the candidate values for the project name.

To merge, identify the correct value.
Then create a new fact that sets the correct value and has all candidates as its prior values.

In [21]:
ProjectName projectAName4 = await j.Fact(new ProjectName(projectA, "Home on Rivercrest", names.ToArray()));

j.RenderFacts(projectAName1, projectAName2, projectAName3, projectAName4)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER --- 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 Construction.Project 
 
 id 
 
 394c7745-5522-4d5c-b... 
 
<!-- RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ==->Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 
 creator 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Cheyenne 
 
 
<!-- zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg== 
 
 Construction.Project.Name 
 
 value 
 
 House on Rivercrest 
 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 E8SmL6Wjyhnfnji88HrGVHnTpanN4qko/p5nWCvfgsAl6Vw/atzQMwYR/UuzkDFLV8Ukxcl01XOKih5nuNPipg==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 
 prior 
 
<!-- 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA== -->
 
 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA== 
 
 Construction.Project.Name 
 
 value 
 
 Home on Cheyenne 
 
 
<!-- 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== -->
 
 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->RBjpGR19b8qeaJdagBiBErY0Q81G9xnKRVXCET8LJ4e3faCdLwltZHAIuNwcVB6LftEmvJVqDp4f8crnZUEobQ== 
 
 
 project 
 
<!-- 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== -->
 
 6it4a+xQBriXTTb1Nc64dp2S4DpzKClYxUUUhgo2Vx0O8GW5yYGTlKBm8bPgh0Z6c0ZdjNSLVQNv+/jyC72shA==->zWE6yfnStZfHlkEl9cC5kuoD0/pLR+0kV8vQcuJ7df5TC83kZX155+dbbgZ87IFQy017tRkgJw8P9yuDJr/eZg== 
 
 
 prior 
 
<!-- +8Tjw3eyaDZgX7/xvL7gse3u2YIpqX/zb433YNKaBMhOr4+b/f2Whp6l2sGMK7BVj4fisWQZRUeyawWvFIzFJA== -->
 
 +8Tjw3eyaDZgX7/xvL7gse3u2YIpqX/zb433YNKaBMhOr4+b/f2Whp6l2sGMK7BVj4fis

And now only one successor matches the specification.

In [22]:
ImmutableList<ProjectName> names = await j.Query(namesOfProject, projectA);

names.Select(n => n.value)

[ Home on Rivercrest ]

## Projections

You can combine specifications to create a *projection*.
This is how you will populate your user interface.

Start your specification as before, and then use the `Select` method to project the results through a new specification.

```csharp
var projectsWithNameCreatedByUser = Given<User>.Match(u =>
    u.Projects().Select(p =>
        new
        {
            p.id,
            Name = p.Names.Select(n => n.name)
        }
    )
);
```

In [23]:
var projectsWithNamesCreatedByUser = Given<User>.Match(u =>
    u.Successors().OfType<Project>(p => p.creator)
        .Where(p => !p.Successors().OfType<ProjectDeleted>(d => d.project).Any(
            d => !d.Successors().OfType<ProjectRestored>(r => r.deleted).Any()))
        .Select(p => new
        {
            ProjectId = p.id,
            Names = p.Successors().OfType<ProjectName>(n => n.project)
                .Where(n => !n.Successors().OfType<ProjectName>(next => next.prior).Any())
                .Select(n => n.value)
        })
);

Using the `Select` method, we defined an anonymous type that contains the project name specification.

In [24]:
var projections = await j.Query(projectsWithNamesCreatedByUser, user);

projections.Select(p => new
{
    p.ProjectId,
    Names = $"[ {string.Join(", ", p.Names)} ]"
}).AsTable()

index,ProjectId,Names
0,394c7745-5522-4d5c-b9ee-2857af844aea,[ Home on Rivercrest ]
1,84d2f319-e553-489f-bf9d-a992ed0be13b,[ ]
2,9b006289-af52-41eb-bde6-20dfb6f953e1,[ ]
